# Gemini Code Documentation Demo
Use free-tier Gemini to annotate code and produce markdown summaries.

## Setup
- Set `GEMINI_API_KEY` in your environment (free tier works).
- Run cells to configure the client.

In [3]:
import os
import textwrap
import google.generativeai as genai

api_key = os.getenv("GEMINI_API_KEY")
if not api_key:
    raise RuntimeError("Set GEMINI_API_KEY before running the notebook.")

genai.configure(api_key=api_key)
model = genai.GenerativeModel("gemini-2.5-flash")
print("Model ready")

Model ready


## Document code with inline comments
Send a prompt asking Gemini to add docstrings and inline comments.

In [4]:
code_snippet = textwrap.dedent(
    """
    def compute_total(items):
        total = 0
        for item in items:
            total += item
        return total
    """
)

doc_prompt = (
    "Add concise docstrings and inline comments to the following code without changing its behaviour. "
    "Return only the updated code block.\n\n"
    f"Code:\n{code_snippet}"
)

doc_response = model.generate_content(doc_prompt)
print(doc_response.text.strip())

```python
def compute_total(items):
    """Calculates the sum of a list of numerical items.

    Args:
        items (list[int | float]): A list of numbers to be summed.

    Returns:
        int | float: The total sum of all items in the list.
    """
    total = 0  # Initialize the total sum to zero
    for item in items:
        total += item  # Add each item to the running total
    return total
```


## Generate a markdown overview
Use a separate prompt to summarise behaviour, inputs, and edge cases.

In [5]:
summary_prompt = (
    "Create a concise markdown summary of the code below. Include sections for Overview, Key Behaviours, "
    "Inputs/Outputs, and Edge Cases. Use bullet points and keep it brief.\n\n"
    f"Code:\n{code_snippet}"
)

summary_response = model.generate_content(summary_prompt)
print(summary_response.text.strip())

```markdown
### Code Summary: `compute_total`

*   **Overview**
    *   Calculates the sum of all numeric elements within an iterable.

*   **Key Behaviours**
    *   Initializes a sum to zero.
    *   Iterates sequentially through each item provided.
    *   Adds each item's value to the running total.
    *   Returns the final accumulated sum.

*   **Inputs/Outputs**
    *   **Input:** `items` (an iterable, expected to contain numeric types like integers or floats).
    *   **Output:** A single numeric value representing the total sum of `items`.

*   **Edge Cases**
    *   **Empty `items` iterable:** Returns `0`.
    *   **Non-numeric items:** Will raise a `TypeError` during addition if an item is not compatible (e.g., adding a string to an integer).
    *   **Single item:** Returns the value of that item.
```


## Generate unit tests
Ask Gemini to propose pytest-style tests that cover behaviours and edge cases.

In [6]:
tests_prompt = (
    "Write pytest-style unit tests for the following code. "
    "Cover common paths and edge cases, keep fixtures minimal, and return only the test module.\n\n"
    f"Code:\n{code_snippet}"
)

tests_response = model.generate_content(tests_prompt)
print(tests_response.text.strip())


```python
import pytest

# Assume the 'compute_total' function is in a module named 'my_module.py'
# For the purpose of this test file, we'll redefine it here,
# but in a real project, you would import it:
# from my_module import compute_total

# --- Start of the code under test (for local execution of tests) ---
def compute_total(items):
    total = 0
    for item in items:
        total += item
    return total
# --- End of the code under test ---


class TestComputeTotal:
    """
    Pytest-style unit tests for the compute_total function.
    Covers common paths and edge cases with minimal fixtures.
    """

    # --- Common Paths ---

    def test_positive_integers(self):
        """Test with a list of positive integers."""
        items = [1, 2, 3, 4, 5]
        assert compute_total(items) == 15

    def test_negative_integers(self):
        """Test with a list of negative integers."""
        items = [-1, -2, -3]
        assert compute_total(items) == -6

    def test_mixed_integ